Regression and Time Series Analysis

In [49]:
from __future__ import print_function, division

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

#packages
#import statsmodels
import statsmodels.formula.api as smf
from collections import Counter
from scipy import stats
import numpy as np
import pandas as pd
import random
import math
import patsy

#thinkstats modules
import thinkstats2
import thinkplot
import scatter
import regression

#thinkstats data
import brfss
import first
import nsfg
import linear

# Chapter 11 Regression

In [5]:
live, firsts, others = first.MakeFrames()

## 11.1 Statsmodel

Running model of Birth weight and Mother's age with statsmodel.

OLS - ordinary least squares

`smf.ols` - Takes in formula string and DataFrame. Returns an OLS object that represent the model

`.fit` - Fit method. fits the model to the data and returns a RegressionRsults object containing results

`.summary` - represents the results in a readable format

In [6]:
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Tue, 22 Feb 2022   Prob (F-statistic):           5.72e-11
Time:                        20:26:41   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Extract the parameters of intercept and slope. 

`results.pvalues['explanatory variable name']` - pvalue of the slope estimate

In [8]:
# Grab attributes of parameters 
# params is a Series that maps variable : parameters

inter = results.params['Intercept']
slope = results.params['agepreg']

inter, slope

(6.830396973311056, 0.017453851471802777)

Estimated parameters are the same as from LeastSquares.

`pvalues` is a Series that maps variable : associated pvalues. To check whether estimated slope is statistically significant

* pvalue for slope is less than 0.001, as expected

In [13]:
slope_pvalue = results.pvalues['agepreg']
slope_pvalue

5.722947107314471e-11

`results.rsquared` = $R^2$, the coefficient of determination

`.f_pvalue` = pvalue associated with model as a whole, similiar to testing whether $R^2$ is statistically significant

`.resid` = sequence of residuals

`.fittedvalues` = sequence of fitted values corresponding to agepreg

In [15]:
results.rsquared

0.004738115474710258

In [16]:
results.f_pvalue

5.722947107255698e-11

In [17]:
results.resid

0        1.403333
1        0.359539
2        2.044489
3       -0.141599
4       -0.962826
           ...   
13581   -0.990532
13584   -0.925080
13588   -0.955495
13591    0.292949
13592    0.292949
Length: 9038, dtype: float64

In [18]:
results.fittedvalues

0        7.409167
1        7.515461
2        7.080511
3        7.141599
4        7.150326
           ...   
13581    7.365532
13584    7.300080
13588    7.142995
13591    7.207051
13592    7.207051
Length: 9038, dtype: float64

## 11.2 Multiple Regression

Saw first babies tend to be light than others, and effect is statistically significant. Is this showing a spurious relationship?

Possible explanation: birth weight dependes on mother's age, and we might expect mothers of first babies are younger than others. 

Steps: 
1. check whether explanation is plausible
2. use multiple regression to investiagte more carefully

terms: 
multiple regression
* a regression with multiple explanatory variables, but only one dependent variable

spurious relationship
* relationship between two variables that is casued by statistically artifact or a factor, not included in the model, that is related to both variables


In [22]:
# difference in birth weight between first and other babies

diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

First babies are 0.125lbs lighter.

In [23]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

mothers of first babies are 3.59 years younger. 

Running the linear model again, we get the change in birth weight as a function of age. 

In [32]:
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']

print(slope)
slope * diff_age

0.017453851471802777


-0.06259709972169447

Slope is 0.0175 lbs/year. If we multiply the slope by the difference in ages, we get the expected difference in birth weight for first babies and others due to mother's age.

Expected difference is 0.063. Age difference plausible explains about half of the difference in weight

**Using Multiple Regression** to explore relationships more systematically. 

Running a single regression with a categorical variable, `isfirst`

In [34]:
# creating new column where True for first babies and False otherwise
live['isfirst'] = live.birthord == 1

# Fit model using isfirst as an explanatory variable (on right side)
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.74
Date:                Wed, 23 Feb 2022   Prob (F-statistic):           2.55e-05
Time:                        13:01:18   Log-Likelihood:                -15909.
No. Observations:                9038   AIC:                         3.182e+04
Df Residuals:                    9036   BIC:                         3.184e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.3259      0.021    356.007      0.000       7.286       7.366
isfirst[T.True]    -0.1248      0.030     -4.212      0.000      -0.183      -0.067
==============================================================================
Omnibus:                      988.919   Durbin-Watson:                   1.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2897.107
Skew:                          -0.589   Prob(JB):                         0.00
Kurtosis:                       5.511   Cond. No.                         2.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From above, 
* Intercept 7.33 (0)
* isfirst[T.True] 0.125 Prob(F-statistic): 2.55e-05
* R^2 0.002

Slope and intercept is statistically significant, which means unlikely to have occurred by chance. But the R^2 value for this model is small, meaning isfirst doesn't account for a substantial part of the variation in birth weight. 

Running a multiple regression!

In [35]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Wed, 23 Feb 2022   Prob (F-statistic):           3.95e-11
Time:                        13:27:45   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000       6.762       7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025      -0.131      -0.009
agepreg             0.0154      0.003      5.499      0.000       0.010       0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* intercept 6.9142 (0)
* isfirst 0.07   P>|t| (0.025)
* agepreg 0.0154 F-statistic (3.95e-11)
* R^2 0.005

In the combined model, 
* parameter for isfirst is smaller bu about half, meaning part of the apparent effect of isfirst is actually accounted for by agepreg when we control for mother's age
* pvalue for isfirst is about 2.5%, which is border statistically significant
* R^2 is little higher, indicating two variables together account for more variation in birth weight than either alone (but not by much)


## 11.3 Nonlinear relationships

Remembering that the contribution of agepreg might be nonlinear.

If we add age squared, we can control for a quadratic relationship between age and weight. 

In [37]:
# create column for squares of ages
live['agepreg2'] = live.agepreg**2

#estimating parameters for agepreg and agepreg2 ==> parabola
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Wed, 23 Feb 2022   Prob (F-statistic):           1.35e-14
Time:                        13:37:50   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     19.937      0.000       5.133       6.252
isfirst[T.True]    -0.0504      0.031     -1.602      0.109      -0.112       0.011
agepreg             0.1124      0.022      5.113      0.000       0.069       0.155
agepreg2           -0.0018      0.000     -4.447      0.000      -0.003      -0.001
==============================================================================
Omnibus:                     1007.149   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3003.343
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                       5.562   Cond. No.                     1.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Apparent effect of isfirst gets even smaller, and is no longer statistically significant. 

Results suggest that the apparent difference in weight between first babies and others might be explained by the difference in mothers' ages, at least in part. 

Conclusion: used regression models for explanation. 
* discovered that an apparent difference in birth weight is actually due to a difference in mother's age. BUt R^2 values of these models still very low. Showing little predictive power. 

Mother's age our 'control variable'
* a variable included in a regression to eliminate or "control for" a spurious relationship

possible to do better with data mining. 

## 11.4 Data mining 

Scenario: Guessing a baby weight at the office betting pool !

Data mining: an approach to finding relationships between variables by testing a large number of models 

Improving out chances of predictive power by finding the most useful variables. 

In [76]:
# Join with respondent data
resp = nsfg.ReadFemResp()

In [77]:
live = live[live.prglngth > 30]
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix= '_r')

Now we can search for variables with explanatory power. 

Because we don't clean most of the variables, we are probably missing some good ones. 

In [44]:
# using patsy

def go_mining(df):
    """ Searches for variables that predict birth weight
    
    df: DataFrame of pregnancy records
    
    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue
            formula = 'totalwgt_lb ~ agepreg +' + name
            
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue
            
            results = model.fit()
        
        except (ValueError, TypeError):
            continue
        
        variables.append((results.rsquared, name))
    
    return variables


For each variable, we construct a model, compute R^2, and append the results to a list. 
* check that each explanatory variable has some variability; otherwise the results of the regression is unreliable. 
* models all include agepreg, since we already know it has some predictive power
* check that num of observations for each model. Variables that contain a large number of nans are not good candidates for prediction
    * `.nobs` = num of observations n


In [45]:
variables = go_mining(join)

## 11.5 Prediction

Next step, sort results and select variables that yield the highest values of R^2

following functions report the variables with the highest values of R^2

In [50]:
import re

def read_variables():
    """ Read Stata dictionary files for NSFG data.
    
    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def mining_report(variables, n=30):
    """Prints variables with highest R^2
    
    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = read_variables()
    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('r$','',name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name,r2)
            

In [51]:
mining_report(variables)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.3008240784470769 LOW BIRTHWEIGHT - BABY 1
prglngth 0.13012519488625052 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.12340041363361054 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.10203149928156041 AGE AT TIME OF CONCEPTION
mosgest 0.02714427463957969 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.01855092529394209 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.016199503586252995
race 0.016199503586252995 RACE
nbrnaliv 0.016017752709788113 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
paydu 0.014003795578114597 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.013430066465713209 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
birthwgt_oz 0.013102457615706165 BD-3 BIRTHWEIGHT IN OUNCES - 1ST BABY FROM THIS PREGNANCY
anynurse 0.012529022541810764 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PRE

Combining the variables that seem to have the most explanatory power.
* `C(race)` - tells formula parser (patsy) to treat race as a categorical variable. encoded here numerically
* `babysex == 1` - 1 for male, 2 for female. writing ==1 converts it to boolean, True for male and False for female
* `nbrnaliv > 1` - is True for multiple births
* `paydu==1` - True for respondents who own their houses
* `totincr` - encoded numerically from 1-14, with each increment representing about $5000 in annual income. So we can treat these values as numerical with units as 5000

In [55]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 +' 
           'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     79.98
Date:                Wed, 23 Feb 2022   Prob (F-statistic):          4.86e-113
Time:                        15:53:37   Log-Likelihood:                -14295.
No. Observations:                8781   AIC:                         2.861e+04
Df Residuals:                    8773   BIC:                         2.866e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                6.6303      0.065    102.223      0.000       6.503       6.757
C(race)[T.2]             0.3570      0.032     11.215      0.000       0.295       0.419
C(race)[T.3]             0.2665      0.051      5.175      0.000       0.166       0.367
babysex == 1[T.True]     0.2952      0.026     11.216      0.000       0.244       0.347
nbrnaliv > 1[T.True]    -1.3783      0.108    -12.771      0.000      -1.590      -1.167
paydu == 1[T.True]       0.1196      0.031      3.861      0.000       0.059       0.180
agepreg                  0.0074      0.003      2.921      0.004       0.002       0.012
totincr                  0.0122      0.004      3.110      0.002       0.005       0.020
==============================================================================
Omnibus:                      398.813   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1388.362
Skew:                          -0.037   Prob(JB):                    3.32e-302
Kurtosis:                       4.947   Cond. No.                         221.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

All variables are statistically significant, but R^2 is only 0.06 (very small).
* RMSE without the model is 1.27lbs
* with the model is 1.23 lbs

Not much of a difference for improved chances of prediction.

## 11.7 Logistic regression

### Implementation

Example using variables in NSFG respondent file to predict whether a baby will be male or female.

Logistic regression using `logit`
* looking for variables that affect the sex ratio

In [57]:
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
live['male'] = (live.babysex==1).astype(int)

Result of `model.fit()` is a BinaryResults object, which is similar to RegressionResults object from `ols`. 

From below: 
* Intercept 0.0058 (0.953)
* agepreg 0.0010   (0.783)
* R^2 6.144e-06

Parameter agepreg is positive 
* suggesting older mothers are more likely to have boys, but high p-value where effect can be eaisly due to chance

R^2 does not apply to logistic regression, but there are several alternatives that are used as "pseudo R^2 values"
* useful for comparing models 


In [60]:
model = smf.logit('male ~ agepreg', data=live)
reuslts = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   male   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8882
Method:                           MLE   Df Model:                            1
Date:                Wed, 23 Feb 2022   Pseudo R-squ.:               6.144e-06
Time:                        16:42:37   Log-Likelihood:                -6156.7
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                    0.7833
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0058      0.098      0.059      0.953      -0.185       0.197
agepreg        0.0010      0.004      0.275      0.783      -0.006       0.009
==============================================================================
"""

More promising variables

In [68]:
formula = 'male ~ agepreg + + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   male   No. Observations:                 8782
Model:                          Logit   Df Residuals:                     8776
Method:                           MLE   Df Model:                            5
Date:                Wed, 23 Feb 2022   Pseudo R-squ.:               0.0001440
Time:                        17:41:24   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6086.3
Covariance Type:            nonrobust   LLR p-value:                    0.8822
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0301      0.104     -0.290      0.772      -0.234       0.173
C(race)[T.2]    -0.0224      0.051     -0.439      0.660      -0.122       0.077
C(race)[T.3]    -0.0005      0.083     -0.005      0.996      -0.163       0.162
agepreg         -0.0027      0.006     -0.484      0.629      -0.014       0.008
hpagelb          0.0047      0.004      1.112      0.266      -0.004       0.013
birthord         0.0050      0.022      0.227      0.821      -0.038       0.048
================================================================================
"""

None of the estimated parameters are statistically significant. The pseudo R^2 value is a little higher, but could be due to chance. 

## 11.9 Accuracy

to make a prediction, need to extract the exogenous and endogenous variables

In [65]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

This sample has more males and female babies. The baseline probability showing as a fraction of probability of male is 0.51

In [66]:
actual = endog['male']
baseline = actual.mean()
baseline

0.507173764518333

If we use the previous model, we can compute the number of predictions we get right. 

accuracy = (sum of true positives + sum of true negatives ) / n

In [69]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1- predict) * (1-actual)
sum(true_pos), sum(true_neg)

(3944.0, 548.0)

accuracy, which is slightly higher than the baseline. 

In [70]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.5115007970849464

To make a prediction for an indiviaul, we have to get their information into a DataFrame.

In [72]:
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
y

0    0.513091
dtype: float64

This person has a 51% chance of having a boy (according to the model).

# Excercises

## Possion Regression

If quantity you want to predict is a count, implement with possion regression `poission` statsmodel function.

In [79]:
#nonlinear model of age

join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2

/var/folders/nd/2hwdbgt95fv2qql2n5f1y09c0000gn/T/ipykernel_11533/1010516777.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  join['age2'] = join.age_r**2


In [84]:
# Solution

formula='numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
formula='numbabes ~ age_r + age2 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.677002
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8877
Method:                           MLE   Df Model:                            6
Date:                Thu, 24 Feb 2022   Pseudo R-squ.:                 0.03686
Time:                        14:43:51   Log-Likelihood:                -14898.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                3.681e-243
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.0324      0.169     -6.098      0.000      -1.364      -0.701
C(race)[T.2]    -0.1401      0.015     -9.479      0.000      -0.169      -0.111
C(race)[T.3]    -0.0991      0.025     -4.029      0.000      -0.147      -0.051
age_r            0.1556      0.010     15.006      0.000       0.135       0.176
age2            -0.0020      0.000    -13.102      0.000      -0.002      -0.002
totincr         -0.0187      0.002     -9.830      0.000      -0.022      -0.015
educat          -0.0471      0.003    -16.076      0.000      -0.053      -0.041
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [85]:
# Solution

columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.496802
dtype: float64

## Multinominal Logistic Regression

If the quantity you want to predict is categorical.

In [86]:
# Solution

# Here's the best model I could find.

formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.084053
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Thu, 24 Feb 2022   Pseudo R-squ.:                  0.1682
Time:                        14:45:41   Log-Likelihood:                -9630.7
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.0156      0.805     11.199      0.000       7.438      10.593
C(race)[T.2]    -0.9237      0.089    -10.418      0.000      -1.097      -0.750
C(race)[T.3]    -0.6179      0.136     -4.536      0.000      -0.885      -0.351
age_r           -0.3635      0.051     -7.150      0.000      -0.463      -0.264
age2             0.0048      0.001      6.103      0.000       0.003       0.006
totincr         -0.1310      0.012    -11.337      0.000      -0.154      -0.108
educat          -0.1953      0.019    -10.424      0.000      -0.232      -0.159
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9570      3.020      0.979      0.328      -2.963       8.877
C(race)[T.2]    -0.4411      0.237     -1.863      0.062      -0.905       0.023
C(race)[T.3]     0.0591      0.336      0.176      0.860      -0.600       0.718
age_r           -0.3177      0.177     -1.798      0.072      -0.664       0.029
age2             0.0064      0.003      2.528      0.011       0.001       0.011
totincr         -0.3258      0.032    -10.175      0.000      -0.389      -0.263
educat          -0.0991      0.048     -2.050      0.040      -0.194      -0.004
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.5238      1.205     -2.924      0.003      -5.886      -1.162
C(race)[T.2]    -0.3213      0.093     -3.445      0.001      -0.504      -0.139
C(race)[T.3]    -0.7706      0.171     -4.509      0.000      -1.106      -0.436
age_r            0.1155      0.071      1.626      0.104      -0.024       0.255
age2            -0.0007      0.001     -0.701      0.483      -0.003       0.001
totincr         -0.2276      0.012    -19.621      0.000      -0.250      -0.205
educat           0.0667      0.017      3.995      0.000       0.034       0.099
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8963      1.305     -2.220      0.026      -5.453      -0.339
C(race)[T.2]    -1.0407      0.104    -10.038      0.000      -1.244      -0.837
C(race)[T.3]    -0.5661      0.156     -3.635      0.000      -0.871      -0.261
age_r            0.2411      0.079      3.038      0.002       0.086       0.397
age2            -0.0035      0.001     -2.977      0.003      -0.006      -0.001
totincr         -0.2932      0.015    -20.159      0.000      -0.322      -0.265
educat          -0.0174      0.021     -0.813      0.416      -0.059       0.025
-

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [87]:
# Solution

# This person has a 75% chance of being currently married, 
# a 13% chance of being "not married but living with opposite 
# sex partner", etc.

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122
